## 문서를 적재하는 벡터DB
앞서 OpenAI의 Embedding API에서는 각 텍스트를 임베딩할 때 파이썬의 Pandas 를
이용하여 각 문서와 임베딩을 적재하고, 그 후 Numpy 를 이용하여 코사인 유사도 식을 직접 구현하여 유
사도를 구했습니다. 하지만 실제 현업에서는 Pandas 가 아닌 각 문서의 임베딩을 적재하기 위한 용도로
특별히 만들어진 도구인 벡터 데이터베이스를 사용하는 경우가 많습니다. 이러한 벡터 데이터베이스로
는 Milvus, Faiss, Chroma 등 다양한 데이터베이스가 있지만 여기서는 가장 손쉽게 사용할 수 있는 벡터
데이터베이스의 예시로 크로마 (Chroma) 와 파이스 (Faiss) 를 소개합니다.


In [ ]:
!pip install langchain-community pypdf chromadb faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.0 MB/s eta 0:

###1.크로마
OpenAI API 키 값을 세팅하기 위한 os, 파일을 다운로드하기 위한 urllib.request를 임포트하고, 실
습을 위해 필요한 랭체인 도구들을 임포트합니다. 앞서 학습했던 PDF 를 로드하는 PyPDFLoader, 문서
들을 다수의 청크로 분할하는 RecursiveCharacterTextSplitter, 청크들을 임베딩 벡터로 변
환 시 OpenAI 의 Embedding API 를 사용하기 위해 OpenAIEmbeddings, 임베딩 벡터들을 적재하기
위한 벡터 데이터베이스인 Chroma와 Faiss를 임포트하고, 사용자의 OpenAI API 키 값을 현재 실습 환
경에 세팅합니다.

In [ ]:
import os
import urllib.request
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
os.environ['OPENAI_API_KEY'] = "Openai_api_key"



2023_ 북한인권보고서.pdf 파일을 외부 코드 저장소로부터 다운로드합니다.


In [ ]:
urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch06/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf", filename="2023_북 한 인 권 보 고 서.pdf")


('2023_북 한 인 권 보 고 서.pdf', <http.client.HTTPMessage at 0x79b16ef39f10>)

이제 랭체인의 PyPDFLoader()를 통해 PDF 파일을 로드합니다. PyPDFLoader(파일명)을 실행하
여 loader라는 객체를 선언하고, 해당 객체를 통해 load_and_split()을 실행하면 PDF 를 여러 개
의 문서 청크로 분할한 문자열 리스트가 반환됩니다.


In [ ]:
loader = PyPDFLoader('2023_북 한 인 권 보 고 서.pdf')
pages = loader.load_and_split()
print('청 크 의 수:', len(pages))


청 크 의 수: 445


이 청크들을 ChatGPT 와 같은 언어 모델들이 처리할 수 있는 적당한 길이로 추가로 분할해봅시다.
RecursiveCharacterTextSplitter()를 이용하여 텍스트를 분할하는 text_splitter 객
체를 만듭니다. 이때 chunk_size의 값을 1000 으로 지정하면 앞으로 text_splitter로 텍스트를
분할할 때 각 분할된 청크는 길이가 1000 을 넘지 않습니다. chunk_overlap은 텍스트를 분할할 때 각
청크가 내용을 얼만큼 겹치게 할 것인지를 정하는 값으로 0 을 사용하면 각 청크의 내용이 겹치지 않습니
다.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

앞서 배운 RecursiveCharacterTextSplitter에서 실습할 때는 파이썬 문자열을
분할하기 위해서 create_documents()를 사용했습니다. 하지만 현재는 PyPDFLoader가 로
드한 각각의 청크는 파이썬 문자열이 아닌 Document(page_content='내용', metadata={
'source': 파일명, 'page': 기존 PDF 파일에서의 페이지 번호})와 같은 형식을 가진 원소
입니다. 문자열이 아닌 위와 같은 형식을 가진 청크들을 text_splitter로 분할하는 경우에는
split_documents()를 사용합니다.

In [ ]:
splitted_docs = text_splitter.split_documents(pages)
print('분 할 된 청 크 의 수:', len(splitted_docs))

분 할 된 청 크 의 수: 496


청크의 수가 445 개에서 496 개로 더 늘어났습니다. 실제로 각 청크의 길이를 재보면 1,000 이 넘지 않는것을 확인할 수 있습니다. 496 개의 청크들에 대해 가장 긴 청크의 길이, 가장 짧은 청크의 길이, 청크들의
평균 길이를 구해봅시다.


In [ ]:
chunks = [splitted_doc.page_content for splitted_doc in splitted_docs]
print('청 크 의 최 대 길 이 :',max(len(chunk) for chunk in chunks))
print('청 크 의 최 소 길 이 :',min(len(chunk) for chunk in chunks))
print('청 크 의 평 균 길 이 :',sum(map(len, chunks))/len(chunks))

청 크 의 최 대 길 이 : 1000
청 크 의 최 소 길 이 : 6
청 크 의 평 균 길 이 : 750.2983870967741


이제 496 개의 청크를 모두 OpenAI 의 Embedding API 로 임베딩하여 크로마 데이터베이스에 적재해봅
시다. 각 청크를 임베딩과 동시에 크로마 데이터베이스에 적재할 때는 Chroma.from_documents(
청크들의 리스트, OpenAIEmbeddings())를 사용합니다. 뒤에서 실습할 파이스 벡터 데이터베이
스도 코드 형식이 거의 동일하므로 기억해둡시다. Chroma.from_documents()를 통해 벡터 데이터
베이스 객체를 만들고 나서 적재된 문서의 수를 출력하는 것은 _collection.count()를 통해 가능
합니다

In [ ]:
db = Chroma.from_documents(splitted_docs, OpenAIEmbeddings(chunk_size=100))
print('문 서 의 수:', db._collection.count())

/tmp/ipython-input-8-3301462923.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  db = Chroma.from_documents(splitted_docs, OpenAIEmbeddings(chunk_size=100))


문 서 의 수: 496


데이터 베이스 객체를 만들고 나서 사용자의 입력과 유사도가 높은 문서들을 찾을때는
similarity_search(사용자의 입력)을 사용 합니다. 북 한인권보고서라는 PDF 파일이므로 ‘북한의 교육 과정’ 이라는 질의를 입력하여 연관 청크들을 찾아봅시다.


In [ ]:
question = '북한의 교육 과정'
docs = db.similarity_search(question)
print('문 서 의 수:', len(docs))

문 서 의 수: 4


연관 청크를 4 개 찾습니다. 실제로 출력하여 ‘북한의 교육 과정’ 과 연관된 문서인지 확인해봅시다

In [ ]:
for doc in docs:
    print(doc)
    print('--' * 10)

page_content='2023 북한인권보고서
40
명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과
서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교
과서가 모든 학생에게 충분히 제공되지 않고 학년을 마치면 다음 학
년에 교과서를 물려주어야 했다는 사례가 다수 수집되었다. 소학교
부터 학교운영비, 꼬마계획 등의 비용을 내야했다는 진술이 꾸준히 
수집되고 있는데, 학교시설 현대화 작업이 진행되면서 학교꾸리기 
비용이 증가했다고 한다. 학교에서 요구하는 돈이나 물품은 교원에 
의해 사실상 강제되고 있었는데, 비용을 내지 못하는 경우 동급생들 
앞에서 망신을 주거나 비판하여 형편이 어려운 학생들은 학교를 그
만두는 선택을 하는 경우가 많다고 한다. 또한 도시와 농촌 간 교육
환경의 차이가 크며 대학입학에서 출신성분에 의한 차별이 있고, 교
육기회의 제공에도 경제력이 영향을 미치고 있어 성분·지역·경제
력에 따른 차별이 존재하는 것으로 나타났다. 교육환경도 열악한데, 
학교시설의 현대화 작업에도 불구하고 양호실, 도서관, 위생시설이 
없는 학교도 많은 것으로 보인다. 교원에 대한 경제적 보상도 적절
히 이루어지지 않아, 교원들은 생계를 유지하기 위해 잘사는 학부모
의 원조를 받거나 자신의 텃밭에 학생을 동원시키고 있어 학생들은 
제대로 된 교육여건을 보장받지 못하고 있는 것으로 나타났다. 또
한, 일반교육보다 정치사상교육을 앞세우고 있으며 교과과정에 실
탄사격을 하는 군사훈련을 편성하여 학생들을 의무적으로 참석하게 
하고 있다.
북한의 사회보장 제도로는 연로연금, 노동능력상실 연금, 유가족 
연금 등 생계가 결핍된 경우 기초적인 생계를 보장하기 위한 연금제
도가 있으며, 사회보험금의 성격을 지닌 보조금 제도가 있다. 연로' metadata={'moddate': '2023-07-31T13:57:54+09:00', 'producer': 'Adobe PDF Library 10.0.1', 'creationdate': '2023-07-31T

북한의 교육과 관련된 문서 4 개가 출력된 것을 확인할 수 있습니다. 크로마 벡터 데이터베이스를 파일
로 저장하는 것도 가능합니다. Chroma.from_documents()에서 persist_directory='디렉
터리명'을 사용합니다. 다음 코드를 수행하면 실제로 코드 실행 경로에 ‘ chroma_test.db’라는 디
렉터리가 생깁니다

In [ ]:
db_to_file = Chroma.from_documents(splitted_docs, OpenAIEmbeddings(chunk_size=100),
                                   persist_directory = './chroma_test.db')
print('문 서 의 수:', db_to_file._collection.count())

문 서 의 수: 992


저장한 데이터베이스 파일을 로드해서 사용해봅시다. 로드는 Chroma(persist_directory='디
렉터리명', embedding_function=사용하고 있는 임베딩))으로 할 수 있습니다.

In [ ]:
db_from_file = Chroma(persist_directory='./chroma_test.db',
                      embedding_function=OpenAIEmbeddings())
print('문 서 의 수:', db_from_file._collection.count())


문 서 의 수: 992


앞서 similarity_search(사용자의 입력)을 사용했을 때는 사용자의 입력에 대해 유사한 청크 4
개를 찾아냈습니다. 내부적으로는 유사도를 구하고 유사도 점수 상위 4 개의 청크를 찾아낸 것입니다.
이번에는 유사한 청크를 상위 3 개만 찾도록 강제하고, 유사도 점수 또한 출력하도록 해보겠습니다.
그러려면 similarity_search_with_relevance_scores(사용자의 입력, k=찾고자 하는
문서의 수)와 같이 하면 됩니다. 유사한 청크를 상위 3 개만 찾도록 강제하기 위해 k 의 값을 3 으로 지
정했고 유사도 점수 상위 3 개의 청크를 찾아서 출력합니다

In [ ]:
question = '북한의 교육 과정'
top_three_docs = db_from_file.similarity_search_with_relevance_scores(question, k=3)
for doc in top_three_docs:
  print(doc)
  print('--' * 10)


(Document(metadata={'total_pages': 448, 'page': 41, 'creationdate': '2023-07-31T13:50:27+09:00', 'creator': 'Adobe InDesign CS6 (Windows)', 'producer': 'Adobe PDF Library 10.0.1', 'page_label': '42', 'source': '2023_북 한 인 권 보 고 서.pdf', 'moddate': '2023-07-31T13:57:54+09:00', 'trapped': '/False'}, page_content='2023 북한인권보고서\n40\n명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과\n서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교\n과서가 모든 학생에게 충분히 제공되지 않고 학년을 마치면 다음 학\n년에 교과서를 물려주어야 했다는 사례가 다수 수집되었다. 소학교\n부터 학교운영비, 꼬마계획 등의 비용을 내야했다는 진술이 꾸준히 \n수집되고 있는데, 학교시설 현대화 작업이 진행되면서 학교꾸리기 \n비용이 증가했다고 한다. 학교에서 요구하는 돈이나 물품은 교원에 \n의해 사실상 강제되고 있었는데, 비용을 내지 못하는 경우 동급생들 \n앞에서 망신을 주거나 비판하여 형편이 어려운 학생들은 학교를 그\n만두는 선택을 하는 경우가 많다고 한다. 또한 도시와 농촌 간 교육\n환경의 차이가 크며 대학입학에서 출신성분에 의한 차별이 있고, 교\n육기회의 제공에도 경제력이 영향을 미치고 있어 성분·지역·경제\n력에 따른 차별이 존재하는 것으로 나타났다. 교육환경도 열악한데, \n학교시설의 현대화 작업에도 불구하고 양호실, 도서관, 위생시설이 \n없는 학교도 많은 것으로 보인다. 교원에 대한 경제적 보상도 적절\n히 이루어지지 않아, 교원들은 생계를 유지하기 위해 잘사는 학부모\n의 원조를 받거나 자신의 텃밭에 학생을 동원시키고 있어 학생들은 \n제대로 된 교육여건을 보장받지 못하고 있는 것으

###2. 파이스
이번에는 크로마 외에 랭체인에서 제공하는 또 다른 벡터 데이터베이스인 파이스를 사용해봅시
다. 앞서 만든 497 개의 청크들을 모두 OpenAI 의 Embedding API 로 임베딩하여 파이스 데이터베
이스에 적재해봅시다. 각 청크를 임베딩과 동시에 크로마 데이터베이스에 적재할 때는 FAISS.
from_documents(청크들의 리스트, OpenAIEmbeddings())를 사용하여 파이스 벡터 데이터
베이스 객체인 faiss_db를 만듭니다. 크로마 벡터 데이터베이스를 사용할 때의 코드가 Chroma.
from_documents(청크들의 리스트, OpenAIEmbeddings())였던 것과 매우 유사합니다. 하
지만 그 외 문서의 수를 확인하는 것, 파일을 저장하고 로드하는 등의 일부 코드는 상이하므로 주의합니
다. 예를 들어 파이스의 경우, 저장된 청크의 수를 확인하고자 할 때는 index.ntotal을 사용합니다.


In [ ]:
faiss_db = FAISS.from_documents(splitted_docs, OpenAIEmbeddings(chunk_size=100))
print('문 서 의 수:', faiss_db.index.ntotal)

문 서 의 수: 496


파이스 벡터 데이터베이스를 파일로 저장하는 것도 가능합니다. 이를 위해서는 faiss_db.
save_local(디렉터리명)을 사 용 합 니 다. 다 음 코 드 를 수 행 하 면 실 제 로 코 드 실 행 경 로 에 ‘
faiss_index’라는 디렉터리가 생깁니다. 반대로 FAISS 의 load_local(디렉터리명)을 사용하
여 저장한 벡터 데이터베이스를 로드할 수 있습니다. 이때 사용한 임베딩을 인자로 알려줘야 하므로
OpenAIEmbeddings()를 전달합니다. allow_dangerous_deserialization은 파이썬 객체
를 저장하거나 전송할 때 사용하는 파일을 읽을 때 일부 파일에 보안 위험이 있을 경우 읽는 것이 거부당
하는 경우가 있는데, 에러를 발생시키지 않고 해당 파일을 신뢰할 수 있으니 무시하고 읽겠다는 의미입니
다. 해당 파일은 방금 전에 사용자가 저장한 것이므로 무시하고 읽도록 True로 설정합니다. 파일을 다시
읽어서 new_db_faiss라는 벡터 데이터베이스 객체에 저장합니다.

In [ ]:
faiss_db.save_local('faiss_index')

new_db_faiss = FAISS.load_local('faiss_index',
                                OpenAIEmbeddings(),
                                allow_dangerous_deserialization=True)

크로마와 마찬가지로 ‘북한의 교육과정’ 으로 검색하여 연관된 문서를 확인해봅시다.

In [ ]:
question = '북한의 교육 과정'
docs = new_db_faiss.similarity_search(question)
for doc in docs:
  print(doc)
  print('--' * 10)



page_content='2023 북한인권보고서
40
명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과
서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교
과서가 모든 학생에게 충분히 제공되지 않고 학년을 마치면 다음 학
년에 교과서를 물려주어야 했다는 사례가 다수 수집되었다. 소학교
부터 학교운영비, 꼬마계획 등의 비용을 내야했다는 진술이 꾸준히 
수집되고 있는데, 학교시설 현대화 작업이 진행되면서 학교꾸리기 
비용이 증가했다고 한다. 학교에서 요구하는 돈이나 물품은 교원에 
의해 사실상 강제되고 있었는데, 비용을 내지 못하는 경우 동급생들 
앞에서 망신을 주거나 비판하여 형편이 어려운 학생들은 학교를 그
만두는 선택을 하는 경우가 많다고 한다. 또한 도시와 농촌 간 교육
환경의 차이가 크며 대학입학에서 출신성분에 의한 차별이 있고, 교
육기회의 제공에도 경제력이 영향을 미치고 있어 성분·지역·경제
력에 따른 차별이 존재하는 것으로 나타났다. 교육환경도 열악한데, 
학교시설의 현대화 작업에도 불구하고 양호실, 도서관, 위생시설이 
없는 학교도 많은 것으로 보인다. 교원에 대한 경제적 보상도 적절
히 이루어지지 않아, 교원들은 생계를 유지하기 위해 잘사는 학부모
의 원조를 받거나 자신의 텃밭에 학생을 동원시키고 있어 학생들은 
제대로 된 교육여건을 보장받지 못하고 있는 것으로 나타났다. 또
한, 일반교육보다 정치사상교육을 앞세우고 있으며 교과과정에 실
탄사격을 하는 군사훈련을 편성하여 학생들을 의무적으로 참석하게 
하고 있다.
북한의 사회보장 제도로는 연로연금, 노동능력상실 연금, 유가족 
연금 등 생계가 결핍된 경우 기초적인 생계를 보장하기 위한 연금제
도가 있으며, 사회보험금의 성격을 지닌 보조금 제도가 있다. 연로' metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2023-07-

보다시피 크로마와 동일한 결과를 얻었습니다. 이렇게 해서 이번 장에서는 랭체인 기초, 텍스트를 청크로
분할하는 방법, 청크를 임베딩 벡터로 변환하고 적재한 후에 사용자의 입력과 유사한 청크를 찾게 도와주
는 벡터 데이터베이스에 대해 알아봤습니다. 앞으로 이어지는 실습에서는 지금까지 랭체인의 다양한 도
구를 이용하여 지금까지보다 더욱 난이도가 높은 AI 서비스들을 개발해보겠습니다.